In [ ]:
#pip install matplotlib
#!pip install seaborn

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn import set_config
import glob
from sklearn.impute import SimpleImputer
import seaborn as sns
import time, pickle
#Preprocessing, Upsampling, Model Selection, Model Evaluation
from sklearn.model_selection import train_test_split, StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score 
from sklearn.metrics import classification_report, confusion_matrix, roc_curve
from imblearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import cross_val_predict, cross_val_score, learning_curve, cross_validate
from sklearn.metrics import fbeta_score, make_scorer, accuracy_score

from sklearn.feature_selection import RFECV

import warnings
warnings.filterwarnings("ignore")
set_config(display='diagram')
print('Imports terminés.')


# Importation des données

In [ ]:
import os
os.chdir('C:\\Users\\hlouk\\OpenClassroom\\P7')
print(os.getcwd())

In [ ]:
print("CHECKLIST DATA EXPLORER :")
display(os.listdir('C:\\Users\\hlouk\\OpenClassroom\\P7'))

In [ ]:
missing_values = ["n/a", "na", "--","nan","NaN"]

In [ ]:
%%time 

dirpath = "C:/Users/hlouk/OpenClassroom/P7/"

application_train = pd.read_csv(dirpath+"application_train.csv")
application_test = pd.read_csv(dirpath+"application_test.csv")
bureau = pd.read_csv(dirpath+"bureau.csv")
bureau_balance = pd.read_csv(dirpath+"bureau_balance.csv")
cash_balance = pd.read_csv(dirpath+"POS_CASH_balance.csv")
card_balance = pd.read_csv(dirpath+"credit_card_balance.csv")
prev_app = pd.read_csv(dirpath+"previous_application.csv")
payments = pd.read_csv(dirpath+"installments_payments.csv")
HomeCredit = pd.read_csv(dirpath+"HomeCredit_columns_description.csv",  na_values = missing_values, encoding='latin1')

Les données sont réparties en 9 fichiers.

  1 fichier principal pour la partie "training" importé dans train_set (1 ligne / prêt)
  1 fichier principal pour la partie "test" importé dans test_set (1 ligne / prêt)
  1 fichier HomeCredit_columns_description propose le descriptif de chaque colonne dans chaque fichier
  6 autres fichiers contenant des informations supplémentaires sur chaque prêt.


# Analyse et pré-traitement

## Train/Test data

In [ ]:
# Training data
print('Training data shape: ', application_train.shape)
application_train.describe()
application_train.head()

In [ ]:
# Testing data
print('application_test shape: ', application_test.shape)
application_test.describe()
application_test.head()

## Autres Fichiers

In [ ]:
# Bureau
print('Bureau data shape: ', bureau.shape)
bureau.head()

In [ ]:
# Bureau balance
print('Bureau balance data shape: ', bureau_balance.shape)
bureau_balance.head()

In [ ]:
# Cash balance
print('Cash balance data shape: ', cash_balance.shape)
cash_balance.head()

In [ ]:
# Card balance
print('Card balance data shape: ', card_balance.shape)
card_balance.head()

In [ ]:
# Previous applications balance
print('Previous applications data shape: ', prev_app.shape)
prev_app.head()

In [ ]:
# Payments balance
print('Payments data shape: ', payments.shape)
payments.head()

In [ ]:
# HomeCredit balance
print('HomeCredit data shape: ', HomeCredit.shape)
HomeCredit.head()

# Analyse des données

In [ ]:
columns_description=pd.read_csv("C:\\Users\\hlouk\\OpenClassroom\\P7\\HomeCredit_columns_description.csv",encoding='latin1')

def describe_sheet(sheet_name):
    '''Display content of HomeCredit_columns_description for sheet_name table.'''
    if 'application' in sheet_name:
        sheet_name = dirpath + 'application_{train|test}'
    description = columns_description[columns_description['Table'].str.contains(sheet_name)].iloc[:,2:]
    description = description.set_index('Row')
    return description
        

def overview(df_name):
    '''Display information, result of describe, shape and head for given df_name.'''
    df = globals()[df_name]
    print('\n\n' + '#'*60)
    print(f'Table {df_name}')
    print('#'*60)
    print(f'\n\nLa table {df_name} a {df.shape[0]:,.0f} lignes et {df.shape[1]:.0f} colonnes.'.replace(',', ' '))
    print(f'\n\nDescription de la table :\n')
    display(describe_sheet(df_name))
    print('\n\nPrincipales statistiques :\n')
    display(df.describe(include='all'))
    print(f'\n\nPremières lignes :\n')
    display(df.head())

## Apercu des Données

In [ ]:
application_train = pd.read_csv(dirpath + 'application_train.csv')

overview('application_train')

## Les valeurs manquantes

In [ ]:
def data_describe(folder):
    '''Check the number of rows, columns, missing values and duplicates.
       Count type of columns.
       Memory indication'''

    data_dict = {}
    for file in folder:
        if file == 'C:/Users/hlouk/OpenClassroom/P7\HomeCredit_columns_description.csv':
            data = pd.read_csv(file,  na_values = missing_values, encoding='latin1')
        else:
            data = pd.read_csv(file)
        data_dict[file] = [data.shape[0], 
                           data.shape[1],
                            round(data.isna().sum().sum()/data.size*100, 2),
                            round(data.duplicated().sum().sum()/data.size*100, 2),
                            data.select_dtypes(include=['object']).shape[1],
                            data.select_dtypes(include=['float']).shape[1],
                            data.select_dtypes(include=['int']).shape[1],
                            data.select_dtypes(include=['bool']).shape[1],
                            round(data.memory_usage().sum()/1024**2, 3)]

        comparative_table = pd.DataFrame.from_dict(data = data_dict, 
                                                   columns = ['Rows', 'Columns', '%NaN', '%Duplicate', 
                                                              'object_dtype','float_dtype', 'int_dtype', 
                                                              'bool_dtype', 'MB_Memory'], 
                                                   orient='index')
    print("SUMMARY FILES…")
    return(comparative_table)

In [ ]:
#Data description  
data_describe(folder = glob.glob('C:/Users/hlouk/OpenClassroom/P7/*.csv'))

In [ ]:
# Fonction qui calcule les valeurs manquantes par colone
def missing_values_table(df):
    # Total missing values
    mis_val = df.isnull().sum()
        
    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)
        
    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)

    # Print some summary information
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
           "There are " + str(mis_val_table_ren_columns.shape[0]) +
           " columns that have missing values.")
        
    # Return the dataframe with missing information
    return mis_val_table_ren_columns

In [ ]:
# Statistiques des valeurs manquantes
missing_values = missing_values_table(application_train)
missing_values.head(20)

In [ ]:
# Nombre de chaque type de colonne
# float64 et int64 pour des variables numériques
# object pour des variables catégorielles
application_train.dtypes.value_counts()

## Les anomalies des valeurs numériques

In [ ]:
def plot_hist_data(df):
    num_cols = [col for col in df.columns if df[col].dtype != 'object'] 
        
    height = int(np.ceil(len(num_cols)/6))
    fig_height = 3 * height
    fig = plt.figure(figsize=(20,fig_height))
    
    for feat_idx, col in enumerate(num_cols):
        ax = fig.add_subplot(height, 6, feat_idx+1)
        ax.hist(df[col], bins=50)
        ax.set_title(col)
    fig.tight_layout(pad=4)  

In [ ]:
def plot_box_data(df):
    num_cols = [col for col in df.columns if df[col].dtype != 'object'] 
    
    height = int(np.ceil(len(num_cols)/6))
    fig_height = 3 * height
    fig = plt.figure(figsize=(20,fig_height))
    
    for feat_idx, col in enumerate(num_cols):
        ax = fig.add_subplot(height, 6, feat_idx+1)
        ax.boxplot(df[col])
        ax.set_title(col)
    fig.tight_layout(pad=4) 

In [ ]:
plot_hist_data(application_train)

In [ ]:
plot_box_data(application_train)

In [ ]:
px.box(application_train['AMT_INCOME_TOTAL'])

In [ ]:
def z_score(array, threshold=3):
    '''Return an array of boolean, True for each value in the array where Z-score >threshold.'''
    mean = array.mean()
    std = array.std()
    return abs((array-mean))/std > threshold

In [ ]:
nb_outliers = application_train[z_score(application_train['AMT_INCOME_TOTAL'])].shape[0]
pc_outliers = nb_outliers / application_train.shape[0]
print(f'Concernant la variable AMT_INCOME_TOTAL, on retrouve {nb_outliers} outliers \
qui représentent {pc_outliers:.2%} des cas.')

In [ ]:
import jinja2
application_train.sort_values(by='AMT_INCOME_TOTAL', ascending=False).head(10).style.format({'AMT_INCOME_TOTAL':'{:,.0f}'})

In [ ]:
#Convertissons-les temporairement en années écoulées (divison par -365) pour faciliter la vérification 

day_cols = [col for col in application_train.columns if "DAYS_" in col]
(application_train[day_cols].describe().loc[['min', 'max']].transpose()/365).style.format({'min':'{:,.2f}', 'max':'{:,.2f}'})

In [ ]:
(application_train[day_cols])[application_train['DAYS_EMPLOYED']>0].describe()

In [ ]:
days_count_pos = application_train['DAYS_EMPLOYED'][application_train['DAYS_EMPLOYED']>0].count()
print(f"Nombre de lignes avec des valeurs aberrantes pour DAYS_EMPLOYED : {days_count_pos} soit {days_count_pos/application_train.shape[0]:.2%} du total")


In [ ]:
# Affiche l'histogramme du nombre de jours employés
fig = plt.figure(1, figsize=(12, 6))
plt.subplot(1, 2, 1)
application_train['DAYS_EMPLOYED'].hist()
plt.xlabel('DAYS_EMPLOYED')
plt.ylabel('Count')
plt.xticks(rotation=30)


In [ ]:
# âge 

for col  in day_cols:
    idx = application_train[application_train[col]<application_train['DAYS_BIRTH']].index
    idx_exist = len(idx)
    if idx_exist:
        print(col, 'présente des valeurs aberrantes aux lignes :', list(idx))
        for i in idx:
            print(application_train.loc[i,[col, 'DAYS_BIRTH']])

In [ ]:
(application_train['DAYS_BIRTH'] / -365).describe()

In [ ]:
#import seaborn as sns
plt.figure(figsize=(20, 5))
sns.histplot((application_train['DAYS_BIRTH'] / -365), bins = 30).set_title('Age of client')


In [ ]:
application_train[application_train['DAYS_EMPLOYED']>0].head()

In [ ]:
application_train['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram', figsize=(20,5));
plt.xlabel('Days Employment')

In [ ]:
print("%0.0f values with 365243 days employed for training data" % 
      len(application_train[application_train['DAYS_EMPLOYED'] == 365243]))
print("%0.0f Total values from days employed for training data" % application_train.shape[0])
print("***********************")
print("%0.0f values with 365243 days employed for testing data" % 
      len(application_test[application_test['DAYS_EMPLOYED'] == 365243]))
print("%0.0f Total values from days employed for testing data" % application_test.shape[0])


In [ ]:
#Create an outliers flag column
application_train['DAYS_EMPLOYED_OUTLIERS'] = application_train["DAYS_EMPLOYED"] == 365243
application_test['DAYS_EMPLOYED_OUTLIERS'] = application_test["DAYS_EMPLOYED"] == 365243

#Replace outliers values with nan
application_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
application_test['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

In [ ]:
application_train['DAYS_EMPLOYED'].plot.hist(title ='Days Employment with preprocessing outliers', figsize=(20,5))
plt.xlabel('Days Employment')

In [ ]:
for col in day_cols:
    application_train.loc[application_train[col]< application_train['DAYS_BIRTH'], col] = application_train['DAYS_BIRTH']
    application_train.loc[application_train[col]>0, col] = 0

### Variables qualitatives

In [ ]:
for col in [col for col in application_train.columns if application_train[col].dtype == 'object']:
    print(col,":", list(application_train[col].unique()))

In [ ]:
def clean_values(df):
    print('Nettoyage des lignes...')
    
    old_length = df.shape[0]
    result = df.copy()
    num_cols = [col for col in result.columns if result[col].dtype!='object']
    cat_cols = [col for col in result.columns if col not in num_cols]
    
    high_z_score = result[num_cols].apply(z_score, axis=0, args=[80])
    result.drop(high_z_score[high_z_score.any(axis=1)].index, inplace=True)
    
    days_cols = [col for col in result.columns if "DAYS_" in col]
    for col in day_cols:
        result.loc[result[col]< result['DAYS_BIRTH'], col] = result['DAYS_BIRTH']
        result.loc[result[col]>0, col] = 0
        
    result[num_cols]=result[num_cols].fillna(0)
    result[cat_cols]=result[cat_cols].fillna('Not specified')
    
    new_length = result.shape[0]
    deleted_rows = old_length - new_length
    print(f'{deleted_rows} lignes ont été supprimées soit {deleted_rows/old_length:.2%} du total.')
  
    return result

In [ ]:
application_train = clean_values(application_train)
application_test = clean_values(application_test)

In [ ]:
def plot_stat(data, feature, title) : 
    
    ax, fig = plt.subplots(figsize=(20,8)) 
    ax = sns.countplot(y=feature, data=data, order=data[feature].value_counts(ascending=False).index)
    ax.set_title(title)

    for p in ax.patches:
                percentage = '{:.1f}%'.format(100 * p.get_width()/len(data[feature]))
                x = p.get_x() + p.get_width()
                y = p.get_y() + p.get_height()/2
                ax.annotate(percentage, (x, y), fontsize=20, fontweight='bold')

def plot_percent_target1(data, feature, title) : 
    
    cat_perc = data[[feature, 'TARGET']].groupby([feature],as_index=False).mean()
    cat_perc.sort_values(by='TARGET', ascending=False, inplace=True)
    
    ax, fig = plt.subplots(figsize=(20,8)) 
    ax = sns.barplot(y=feature, x='TARGET', data=cat_perc)
    ax.set_title(title)
    ax.set_xlabel("")
    ax.set_ylabel("Percent of target with value 1")

    for p in ax.patches:
                percentage = '{:.1f}%'.format(100 * p.get_width())
                x = p.get_x() + p.get_width()
                y = p.get_y() + p.get_height()/2
                ax.annotate(percentage, (x, y), fontsize=20, fontweight='bold')



In [ ]:
ax, fig = plt.subplots(figsize=(20,8)) 
ax = sns.countplot(y='TARGET', data=application_train)
ax.set_title("Distribution du TARGET")

for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * p.get_width()/len(application_train.TARGET))
        x = p.get_x() + p.get_width()
        y = p.get_y() + p.get_height()/2
        ax.annotate(percentage, (x, y), fontsize=20, fontweight='bold')


In [ ]:

#NAME_CONTRACT_TYPE
plot_stat(application_train, 'NAME_CONTRACT_TYPE',"Type of contract")
print("                                   -------------------------------------------------------")
plot_percent_target1(application_train, 'NAME_CONTRACT_TYPE',"Type of contract %Target1")


In [ ]:
#CODE_GENDER
plot_stat(application_train, 'CODE_GENDER',"GENDER Distribution")
print("                                   -------------------------------------------------------")
plot_percent_target1(application_train, 'CODE_GENDER',"GENDER Distribution %Target1")

In [ ]:
#FLAG_OWN_CAR
plot_stat(application_train, 'FLAG_OWN_CAR',"Car owner")
print("                                   -------------------------------------------------------")
plot_percent_target1(application_train, 'FLAG_OWN_CAR',"Car owner %Target1")


In [ ]:
#NAME_FAMILY_STATUS
plot_stat(application_train, 'NAME_FAMILY_STATUS',"Family status")
print("                                   -------------------------------------------------------")
plot_percent_target1(application_train, 'NAME_FAMILY_STATUS',"Family status %Target1")

In [ ]:
#NAME_INCOME_TYPE
plot_stat(application_train, 'NAME_INCOME_TYPE',"Income type of client")
print("                                   -------------------------------------------------------")
plot_percent_target1(application_train, 'NAME_INCOME_TYPE',"Income type of client %Target1")

In [ ]:
#OCCUPATION_TYPE
plot_stat(application_train, 'OCCUPATION_TYPE',"Ocupation of client")
print("                                   -------------------------------------------------------")
plot_percent_target1(application_train, 'OCCUPATION_TYPE',"Ocupation of client %Target1")

In [ ]:
#NAME_EDUCATION_TYPE
plot_stat(application_train, 'NAME_EDUCATION_TYPE',"Education type of the client")
print("                                   -------------------------------------------------------")
plot_percent_target1(application_train, 'NAME_EDUCATION_TYPE',"Education type of the client %Target1")

In [ ]:
#NAME_HOUSING_TYPE
plot_stat(application_train, 'NAME_HOUSING_TYPE',"Type of the housing of client")
print("                                   -------------------------------------------------------")
plot_percent_target1(application_train, 'NAME_HOUSING_TYPE',"Type of the housing of client %Target1")

# Method #1: Modelization

In [ ]:
# HOME CREDIT DEFAULT RISK COMPETITION
# Most features are created by applying min, max, mean, sum and var functions to grouped tables. 
# Little feature selection is done and overfitting might be a problem since many features are related.
# The following key ideas were used:
# - Divide or subtract important features to get rates (like annuity and income)
# - In Bureau Data: create specific features for Active credits and Closed credits
# - In Previous Applications: create specific features for Approved and Refused applications
# - Modularity: one function for each table (except bureau_balance and application_test)
# - One-hot encoding for categorical features
# All tables are joined with the application DF using the SK_ID_CURR key (except bureau_balance).
# You can use LightGBM with KFold or Stratified KFold.

# Update 16/06/2018:
# - Added Payment Rate feature
# - Removed index from features
# - Use standard KFold CV (not stratified)

import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import json ,lightgbm
import re


@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

# Preprocess application_train.csv and application_test.csv
def application_train_test(num_rows = None, nan_as_category = False):
    # Read data and merge
    df = pd.read_csv('../P7/application_train.csv', nrows= num_rows)
    test_df = pd.read_csv('../P7/application_test.csv', nrows= num_rows)
    print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))
    df = df.append(test_df).reset_index()
    # Optional: Remove 4 applications with XNA CODE_GENDER (train set)
    df = df[df['CODE_GENDER'] != 'XNA']
    
    # Categorical features with Binary encode (0 or 1; two categories)
    '''for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])'''
    # Categorical features with One-Hot encode
    df, cat_cols = one_hot_encoder(df, nan_as_category)
    
    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    
    # Some simple new features (percentages)
    

    #INCOME_CREDIT_PERC: Pourcentage du montant du crédit par rapport au revenu d'un client
    #ANNUITY_INCOME_PERC: Pourcentage de la rente de prêt par rapport au revenu d'un client
    #DAYS_EMPLOYED_PERC: Pourcentage des jours employés par rapport à l'âge du client
       
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    del test_df
    gc.collect()
    return df

# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance(num_rows = None, nan_as_category = True):
    bureau = pd.read_csv('../P7/bureau.csv', nrows = num_rows)
    bb = pd.read_csv('../P7/bureau_balance.csv', nrows = num_rows)
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)
    
    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    del bb, bb_agg
    gc.collect()
    
    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    del closed, closed_agg, bureau
    gc.collect()
    return bureau_agg

# Preprocess previous_applications.csv
def previous_applications(num_rows = None, nan_as_category = True):
    prev = pd.read_csv('../P7/previous_application.csv', nrows = num_rows)
    prev, cat_cols = one_hot_encoder(prev, nan_as_category= True)
    # Days 365.243 values -> nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    # Add feature: value ask / value received percentage
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    # Previous applications numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }
    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    return prev_agg

# Preprocess POS_CASH_balance.csv
def pos_cash(num_rows = None, nan_as_category = True):
    pos = pd.read_csv('../P7/POS_CASH_balance.csv', nrows = num_rows)
    pos, cat_cols = one_hot_encoder(pos, nan_as_category= True)
    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    # Count pos cash accounts
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    del pos
    gc.collect()
    return pos_agg
    
# Preprocess installments_payments.csv
def installments_payments(num_rows = None, nan_as_category = True):
    ins = pd.read_csv('../P7/installments_payments.csv', nrows = num_rows)
    ins, cat_cols = one_hot_encoder(ins, nan_as_category= True)
    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    # Count installments accounts
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    del ins
    gc.collect()
    return ins_agg

# Preprocess credit_card_balance.csv
def credit_card_balance(num_rows = None, nan_as_category = True):
    cc = pd.read_csv('../P7/credit_card_balance.csv', nrows = num_rows)
    cc, cat_cols = one_hot_encoder(cc, nan_as_category= True)
    # General aggregations
    cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    # Count credit card lines
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
    del cc
    gc.collect()
    return cc_agg

# LightGBM GBDT with KFold or Stratified KFold
# Parameters from Tilii kernel: https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code
def kfold_lightgbm(df, num_folds, stratified = False, debug= False):
    # Divide in training/validation and test data
    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]
    train_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in train_df.columns]
    test_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in test_df.columns]
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    del df
    gc.collect()
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=1001)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=1001)
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]
        

        # LightGBM parameters found by Bayesian optimization
        clf = LGBMClassifier(
            nthread=4,
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=34,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1,
            verbose=-1 )

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
            eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)

        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

    print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    # Write submission file and plot feature importance
    if not debug:
        test_df['TARGET'] = sub_preds
        test_df[['SK_ID_CURR', 'TARGET']].to_csv(submission_file_name, index= False)
    display_importances(feature_importance_df)
    return feature_importance_df


In [ ]:
# Display/plot feature importance

def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances01.png')


def main(debug = False):
    num_rows = 10000 if debug else None
    df = application_train_test(num_rows)
    with timer("Process bureau and bureau_balance"):
        bureau = bureau_and_balance(num_rows)
        print("Bureau df shape:", bureau.shape)
        df = df.join(bureau, how='left', on='SK_ID_CURR')
        del bureau
        gc.collect()
    with timer("Process previous_applications"):
        prev = previous_applications(num_rows)
        print("Previous applications df shape:", prev.shape)
        df = df.join(prev, how='left', on='SK_ID_CURR')
        del prev
        gc.collect()
    with timer("Process POS-CASH balance"):
        pos = pos_cash(num_rows)
        print("Pos-cash balance df shape:", pos.shape)
        df = df.join(pos, how='left', on='SK_ID_CURR')
        del pos
        gc.collect()
    with timer("Process installments payments"):
        ins = installments_payments(num_rows)
        print("Installments payments df shape:", ins.shape)
        df = df.join(ins, how='left', on='SK_ID_CURR')
        del ins
        gc.collect()
    with timer("Process credit card balance"):
        cc = credit_card_balance(num_rows)
        print("Credit card balance df shape:", cc.shape)
        df = df.join(cc, how='left', on='SK_ID_CURR')
        del cc
        gc.collect()
    with timer("Run LightGBM with kfold"):
        feat_importance = kfold_lightgbm(df, num_folds= 10, stratified= False, debug= debug)


In [ ]:
if __name__ == "__main__":
    submission_file_name = "submission_kernel02.csv"
    with timer("Full model run"):
        main()

# Method #2: Modelization

In [ ]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import json ,lightgbm
import re


@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    #df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

# Preprocess application_train.csv and application_test.csv
def application_train_test(num_rows = None, nan_as_category = False):
    # Read data and merge
    df = pd.read_csv('../P7/application_train.csv', nrows= num_rows)
    test_df = pd.read_csv('../P7/application_test.csv', nrows= num_rows)
    print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))
    df = df.append(test_df).reset_index()

    # Optional: Remove 4 applications with XNA CODE_GENDER (train set)
    df = df[df['CODE_GENDER'] != 'XNA']
    
    # Categorical features with Binary encode (0 or 1; two categories)
    '''for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])'''
    # Categorical features with One-Hot encode
    #df, cat_cols = one_hot_encoder(df, nan_as_category)
    
    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    
    #df['DAYS_BIRTH'] = df[df['DAYS_BIRTH']/ -365]
    #(application_train['DAYS_BIRTH'] / -365).describe()
    
    # Some simple new features (percentages)
    

    #INCOME_CREDIT_PERC: Pourcentage du montant du crédit par rapport au revenu d'un client
    #ANNUITY_INCOME_PERC: Pourcentage de la rente de prêt par rapport au revenu d'un client
    #DAYS_EMPLOYED_PERC: Pourcentage des jours employés par rapport à l'âge du client
       
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    del test_df
    gc.collect()
    return df

# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance(num_rows = None, nan_as_category = True):
    bureau = pd.read_csv('../P7/bureau.csv', nrows = num_rows)
    bb = pd.read_csv('../P7/bureau_balance.csv', nrows = num_rows)
    
    
    #Total number of previous credits taken by each customer
    previous_loan_counts = bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(
                                       columns = {'SK_ID_BUREAU': 'PREVIOUS_LOANS_COUNT'})

#Monthly average balances of previous credits in Credit Bureau.
    bureau_bal_mean = bb.groupby('SK_ID_BUREAU', as_index=False).mean().rename(columns = 
                                        {'MONTHS_BALANCE': 'MONTHS_BALANCE_MEAN'})
    bureau_full = bureau.merge(bureau_bal_mean, on='SK_ID_BUREAU', how='left')

    bureau_mean = bureau_full.groupby('SK_ID_CURR', as_index=False).mean().add_prefix('PREV_BUR_MEAN_')
    bureau_mean = bureau_mean.rename(columns = {'PREV_BUR_MEAN_SK_ID_CURR' : 'SK_ID_CURR'})

    
    # Bureau and bureau_balance numeric features
    return previous_loan_counts, bureau_mean

# Preprocess previous_applications.csv
def previous_applications(num_rows = None, nan_as_category = True):
    prev = pd.read_csv('../P7/previous_application.csv', nrows = num_rows)
    #prev, cat_cols = one_hot_encoder(prev, nan_as_category= True)
    # Days 365.243 values -> nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    # Add feature: value ask / value received percentage
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    
     #Number of previous applications of the clients to Home Credit
    previous_application_counts = prev.groupby('SK_ID_CURR', 
                                                           as_index=False)['SK_ID_PREV'].count().rename(
                                                           columns = {'SK_ID_PREV': 'PREVIOUS_APPLICATION_COUNT'})
    
    # Previous applications numeric features
    return previous_application_counts, prev

# Preprocess POS_CASH_balance.csv
def pos_cash(num_rows = None, nan_as_category = True):
    pos = pd.read_csv('../P7/POS_CASH_balance.csv', nrows = num_rows)
    
    POS_mean = pos.groupby('SK_ID_PREV', as_index=False).mean().add_prefix('POS_MEAN_')
    POS_mean.rename(columns = {'POS_MEAN_SK_ID_PREV' : 'SK_ID_PREV'}, inplace=True)

    return POS_mean
    
# Preprocess installments_payments.csv
def installments_payments(num_rows = None, nan_as_category = True):
    ins = pd.read_csv('../P7/installments_payments.csv', nrows = num_rows)
    #ins, cat_cols = one_hot_encoder(ins, nan_as_category= True)
    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    
    install_pay_mean = ins.groupby('SK_ID_PREV', as_index=False).mean().add_prefix('INSTALL_MEAN_')
    install_pay_mean.rename(columns = {'INSTALL_MEAN_SK_ID_PREV' : 'SK_ID_PREV'}, inplace=True)

    return install_pay_mean

# Preprocess credit_card_balance.csv
def credit_card_balance(num_rows = None, nan_as_category = True):
    cc = pd.read_csv('../P7/credit_card_balance.csv', nrows = num_rows)
    #cc, cat_cols = one_hot_encoder(cc, nan_as_category= True)
    # General aggregations
    #cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
    credit_card_balance_mean = cc.groupby('SK_ID_PREV', as_index=False).mean().add_prefix('CARD_MEAN_')
    credit_card_balance_mean.rename(columns = {'CARD_MEAN_SK_ID_PREV' : 'SK_ID_PREV'}, inplace=True)

    return credit_card_balance_mean


## LGBM- Pre tuning

In [ ]:
num_rows = 40000 #if debug else None
df = application_train_test(num_rows)

with timer("Process bureau and bureau_balance"):
        previous_loan_counts, bureau_mean = bureau_and_balance(num_rows)
        print("Bureau df shape:", bureau.shape)
        df = df.merge(previous_loan_counts, on='SK_ID_CURR', how='left')
        df = df.merge(bureau_mean, on='SK_ID_CURR', how='left')
        del previous_loan_counts, bureau_mean
        
with timer("Process previous_applications"):
        prev, prev2 = previous_applications(num_rows)
        print("Previous applications df shape:", prev.shape)
        df.merge(prev, on='SK_ID_CURR', how='left')
        del prev
        
with timer("Process POS-CASH balance"):
        pos = pos_cash(num_rows)
        print("Pos-cash balance df shape:", pos.shape)
        prev2 = prev2.merge(pos, on='SK_ID_PREV', how='left')
        del pos
        
with timer("Process installments payments"):
        ins = installments_payments(num_rows)
        print("Installments payments df shape:", ins.shape)
        #df = df.join(ins, how='left', on='SK_ID_CURR')
        prev2 = prev2.merge(ins, on='SK_ID_PREV', how='left')
        del ins
        
with timer("Process credit card balance"):
        cc = credit_card_balance(num_rows)
        print("Credit card balance df shape:", cc.shape)
        df = df.join(cc, how='left', on='SK_ID_CURR')
        prev2= prev2.merge(cc, on='SK_ID_PREV', how='left')
        del cc
        
'''    with timer("Run LightGBM with kfold"):
        feat_importance = kfold_lightgbm(df, num_folds= 10, stratified= False, debug= debug)
'''
prev_appl_mean = prev2.groupby('SK_ID_CURR', as_index=False).mean().add_prefix('PREV_APPL_MEAN_')
prev_appl_mean.rename(columns = {'PREV_APPL_MEAN_SK_ID_CURR' : 'SK_ID_CURR'}, inplace=True)
prev_appl_mean = prev_appl_mean.drop('PREV_APPL_MEAN_SK_ID_PREV', axis=1)


#Last merge with our data sample
df= df.merge(prev_appl_mean, on='SK_ID_CURR', how='left')

df.head()
df.shape

In [ ]:
import seaborn as sns
import time, pickle
from zipfile import ZipFile
#Preprocessing, Upsampling, Model Selection, Model Evaluation
from sklearn.model_selection import train_test_split, StratifiedKFold
from contextlib import contextmanager
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score 
from sklearn.metrics import classification_report, confusion_matrix, roc_curve
from imblearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import cross_val_predict, cross_val_score, learning_curve, cross_validate
from sklearn.feature_selection import RFECV
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import json ,lightgbm
from sklearn.metrics import recall_score

### Remove N/A columns and values

In [ ]:
#Global view of the missing values (black)
plt.figure(figsize=(20,10))
sns.heatmap(df.notna(), cbar=False)
#show()

In [ ]:
def nan_check(data):
    '''Check Missing Values'''
    total = data.isnull().sum()
    percent_1 = data.isnull().sum()/data.isnull().count()*100
    percent_2 = (np.round(percent_1, 2))
    missing_data = pd.concat([total, percent_2], 
                             axis=1, keys=['Total', '%']).sort_values('%', ascending=False)
    missing_data = missing_data.reset_index()
    missing_data.columns = ['Col_Name','Total', '%']
    return missing_data

print('TOP 20 Missing values from Training dataset')
nan_check(df)[:20]

In [ ]:
print("Nb of columns in the training sample: " + str(len(nan_check(df))))

print("Nb of columns in the training sample with missing values: " + 
      str(len(nan_check(df)[nan_check(df)['Total']!=0])))

print("Nb of columns in the training sample with 55% missing values: " + 
      str(len(nan_check(df)[nan_check(df)['%']>=55])))

In [ ]:
keepCol = nan_check(df)[nan_check(df)['%']<=55]
column_keep = list(keepCol['Col_Name'].values)

#Nouvelle liste avec les colonnes conservées
df = df[column_keep]
df.shape

In [ ]:
#Global view of the missing values (black)
plt.figure(figsize=(20,10))
sns.heatmap(df.notna(), cbar=False)
#show()

### Encode Categorical

In [ ]:
df2, cat_cols = one_hot_encoder(df, nan_as_category= False)


In [ ]:
print(df2.shape)
cat_cols

In [ ]:
df2=df2.drop('FLAG_DOCUMENT_12', axis=1)

In [ ]:
#Display correlations with features engineering
print('Most Positive Correlations:\n', df2.corr()['TARGET'].sort_values().tail(15))
print("--------------------------")
print('Most Negative Correlations:\n', df2.corr()['TARGET'].sort_values().head(15))

In [ ]:
data_train = df2[df2['SK_ID_CURR'].isin(application_train.SK_ID_CURR)]
data_test = df2[df2['SK_ID_CURR'].isin(application_test.SK_ID_CURR)]

data_test = data_test.drop('TARGET', axis=1)

In [ ]:
data_train.set_index('SK_ID_CURR', inplace=True)
data_test.set_index('SK_ID_CURR', inplace=True)

In [ ]:
print('Training Features shape with categorical columns: ', data_train.shape)
print('Testing Features shape with categorical columns: ', data_test.shape)
print('Full data Features shape with categorical columns: ', df2.shape)

### Missing data imputation

In [ ]:
print('Missing values in train data: ', sum(data_train.isnull().sum()))
print('Missing values in test data: ', sum(data_test.isnull().sum()))
print('Missing values in full data: ', sum(df2.isnull().sum()))

In [ ]:
imputer = SimpleImputer(strategy='median')
data_train.iloc[:,:] = imputer.fit_transform(data_train)

In [ ]:
imputer = SimpleImputer(strategy='median')
data_test.iloc[:,:] = imputer.fit_transform(data_test)

In [ ]:
#Check Imputation step
print(sum(data_train.isnull().sum()))

## Modeling Primaire

In [ ]:
TARGET = data_train['TARGET']

In [ ]:
TARGET.to_csv('TARGET.csv', index=True)

In [ ]:
feats = [f for f in data_train.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
data_train = data_train[feats]
data_test = data_test[feats]

In [ ]:
print('Training Features shape with categorical columns: ', data_train.shape)
print('Testing Features shape with categorical columns: ', data_test.shape)

In [ ]:
categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
categorical_columns

In [ ]:
#Nouvelle liste avec les colonnes conservées
df_orig = df[categorical_columns]
df_orig['SK_ID_CURR'] = df['SK_ID_CURR']
df_orig = df_orig[df_orig['SK_ID_CURR'].isin(application_train.SK_ID_CURR)]

df_orig.shape

In [ ]:
default_risk = data_train.merge(df_orig, on='SK_ID_CURR', how='left')
default_risk.shape

In [ ]:
%%time
compression_opts = dict(method='zip', archive_name='default_risk.csv')
default_risk.to_csv('default_risk.zip', index=True, compression=compression_opts)


In [ ]:
from sklearn.model_selection import train_test_split

X_data =None
X_train, X_test, y_train, y_test = train_test_split(data_train, TARGET.values, test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
y_test.sum()

### Standardization

In [ ]:
from sklearn.preprocessing import MinMaxScaler
#Fit scaler to our training data
scaler = MinMaxScaler(feature_range = (0, 1))
scaler.fit(data_train)
scaled_train = scaler.transform(data_train)

In [ ]:
scaler = MinMaxScaler(feature_range = (0, 1))
scaler.fit(data_test)
scaled_test = scaler.transform(data_test)

## Confustion Matrix and AUC

In [ ]:
def cf_matrix_roc_auc(model, y_true, y_pred, y_pred_proba, feature_importances):
    '''This function will make a pretty plot of 
  an sklearn Confusion Matrix using a Seaborn heatmap visualization + ROC Curve.'''
    fig = plt.figure(figsize=(20,15))
  
    plt.subplot(221)
    cf_matrix = confusion_matrix(y_true, y_pred)
    group_names = ['True Neg', 'False Pos', 'False Neg', 'True Pos']
    group_counts = ["{0:0.0f}".format(value) for value in cf_matrix.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in cf_matrix.flatten()/np.sum(cf_matrix)]
  
    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    sns.heatmap(cf_matrix, annot=labels, fmt="", cmap='Blues')

    plt.subplot(222)
    fpr,tpr,_ = roc_curve(y_true, y_pred_proba)
    plt.plot(fpr, tpr, color='orange', linewidth=5, label='AUC = %0.4f' %roc_auc)
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()

    if feature_importances and X_data is not None : 
        plt.subplot(212)
        indices = np.argsort(model.feature_importances_)[::-1]
    
    features = []
    if X_data is not None:
        for i in range(20):
            features.append(X_data.columns[indices[i]]) #After RFECV selection > X_data created

        sns.barplot(x=features, y=model.feature_importances_[indices[range(20)]], color=("orange"))
        plt.xlabel('Features importance')
        plt.xticks(rotation=90)


In [ ]:
#Predictive Model
from lightgbm import LGBMClassifier
from sklearn.metrics import fbeta_score

models = [
          LGBMClassifier(device='gpu')
          ]

In [ ]:
def train_models(model, X_train, X_test, y_train, y_test):
    X_train = X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    model.fit(X_train, y_train)
    output = {
      'AUC': roc_auc_score(y_test, model.predict_proba(X_test)[:,1]),
      'Accuracy': accuracy_score(y_test, model.predict(X_test)),
      'Precision': precision_score(y_test, model.predict(X_test)),
      'Recall': recall_score(y_test, model.predict(X_test)),
      'F1': f1_score(y_test, model.predict(X_test)),
      'F2': fbeta_score(y_test, model.predict(X_test), beta=2, average='binary')
      }
          
    return output

In [ ]:
%%time

name = []
auc = []
accuracy = []
precision = []
recall = []
f1 = []
f2 = []
time_ = []

for model in models:
    start = time.time()    
    results = train_models(model, X_train, X_test, y_train, y_test)

    name.append(type(model).__name__)
    auc.append(results['AUC'])
    accuracy.append(results['Accuracy'])
    precision.append(results['Precision'])
    recall.append(results['Recall'])
    f1.append(results['F1'])
    f2.append(results['F2'])
    time_.append(time.time()-start)

#Initialise data of lists
base_models = pd.DataFrame(data=[name, auc, accuracy, precision, recall, f1, f2, time_]).T
base_models.columns = ['Model', 'AUC', 'Accuracy', 'Precision', 'Recall', 'F1', 'F2', 'Time']
base_models.sort_values('AUC', ascending=False, inplace=True)

In [ ]:
base_models

In [ ]:
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:,1])
cf_matrix_roc_auc(model, y_test, model.predict(X_test), model.predict_proba(X_test)[:,1], feature_importances=None)

##  Using SMOTE

In [ ]:
# SMOTE with Imbalance Data using imblearn module
from imblearn.over_sampling import SMOTE
print("Label 1, Before using SMOTE: {} ".format(sum(y_train==1)))
print("Label 0, Before using SMOTE: {} ".format(sum(y_train==0)))

In [ ]:
%%time
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

In [ ]:
print("Label 1, After using SMOTE: {}".format(sum(y_train_res==1)))
print("Label 0, After using SMOTE: {}".format(sum(y_train_res==0)))

In [ ]:
%%time

name = []
auc = []
accuracy = []
precision = []
recall = []
f1 = []
f2 = []
time_ = []

for model in models:
    start = time.time()    
    results = train_models(model, X_train_res, X_test, y_train_res, y_test)

    name.append(type(model).__name__)
    auc.append(results['AUC'])
    accuracy.append(results['Accuracy'])
    precision.append(results['Precision'])
    recall.append(results['Recall'])
    f1.append(results['F1'])
    f2.append(results['F2'])
    time_.append(time.time()-start)

#Initialise data of lists
base_models_sm = pd.DataFrame(data=[name, auc, accuracy, precision, recall, f1, f2, time_]).T
base_models_sm.columns = ['Model', 'AUC', 'Accuracy', 'Precision', 'Recall', 'F1', 'F2','Time']
base_models_sm.sort_values('AUC', ascending=False, inplace=True)

In [ ]:
base_models_sm

In [ ]:
for model in models:
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:,1])
    cf_matrix_roc_auc(model, y_test, model.predict(X_test), model.predict_proba(X_test)[:,1], feature_importances=False)

## Feature Selection 

In [ ]:
%%time
start = time.time()
rfecv = RFECV(estimator=LGBMClassifier(objective='binary'), 
              step=5, 
              cv=StratifiedKFold(5), 
              scoring='roc_auc', 
              verbose=2,
              n_jobs=-1)
rfecv.fit(X_train, y_train)

print('Time Elapsed: {}'.format(time.time()-start))

In [ ]:
print("Optimal number of features : %d" % rfecv.n_features_)
print("Selected Features: %s" % rfecv.support_[:20])
print("Feature Ranking : %s" % rfecv.ranking_[:20])

In [ ]:
ranking = pd.DataFrame({'Features': data_train.columns})
ranking['RANK'] = np.asarray(rfecv.ranking_)
ranking.sort_values('RANK', inplace=True)

features_selection = ranking[ranking.RANK == 1]['Features'].to_list()
print(len(features_selection))
print(features_selection)

In [ ]:
#Plot number of features vs CV scores
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.feature_selection import RFE, RFECV
step=5
plt.figure(figsize=(20, 5))
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.xlabel('Number of features tested (x%d)' % step)
plt.ylabel('Cross-validation score (roc_auc)')
#show()

In [ ]:
test=ranking.copy()
test['Row'] =test.reset_index().index
test.head()

In [ ]:
print(test.shape)

In [ ]:
test.to_csv('features_description.csv', index=True)

In [ ]:
X_data = data_train.loc[:, data_train.columns[rfecv.get_support()]]
print(X_data.shape)
X_test = data_test.loc[:, data_test.columns[rfecv.get_support()]]
print(X_test.shape)

In [ ]:
X_test.to_csv('X_test.csv', index=True)

In [ ]:
#Merge operation with TARGET before save this reduced sample
X_data = X_data.merge(TARGET, left_index=True, right_index=True)
X_data.shape

In [ ]:
%%time
compression_opts = dict(method='zip', archive_name='X_data.csv')
X_data.to_csv('X_data.zip', index=True, compression=compression_opts)

In [ ]:
#Transform data with rfecv 
X_train_selected = rfecv.transform(X_train)
X_test_selected = rfecv.transform(X_test)

In [ ]:
# random forest Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import fbeta_score

rf_basic = RandomForestClassifier(n_estimators=1000)
#rf_basic.max_depth = 150
# Training the basic Decision Tree model with training set 
rf_basic.fit(X_train_selected, y_train)

# Predicting the output of the test cases using the algorithm created above
y_pre = rf_basic.predict(X_test_selected)

# Validating the algorithm using various Performance metrics

a1 = accuracy_score(y_test,y_pre)
f1 = f1_score(y_test, y_pre, average="macro")
f2 = fbeta_score(y_test, y_pre, beta=2, average="binary")
p1 = precision_score(y_test, y_pre, average="macro")
r1 = recall_score(y_test, y_pre, average="macro")
print("accuracy score : ",a1)
print("f1 score : ",f1)
print("f2 score : ",f2)
print("precision score : ",p1)
print("recall score : ",r1)

# Computing Confusion matrix for the above algorithm

cnf_matrix = confusion_matrix(y_test, y_pre)
np.set_printoptions(precision=2)


In [ ]:
roc_auc = roc_auc_score(y_test, rf_basic.predict_proba(X_test_selected)[:,1])
cf_matrix_roc_auc(rf_basic, y_test, rf_basic.predict(X_test_selected), rf_basic.predict_proba(X_test_selected)[:,1], feature_importances=True)

In [ ]:
#use reduced features with decision tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import fbeta_score


dectree_basic = DecisionTreeClassifier()
dectree_basic.max_depth = 150
# Training the basic Decision Tree model with training set 
dectree_basic.fit(X_train_selected, y_train)

# Predicting the output of the test cases using the algorithm created above
y_pre = dectree_basic.predict(X_test_selected)

# Validating the algorithm using various Performance metrics

a1 = accuracy_score(y_test,y_pre)
f1 = f1_score(y_test, y_pre, average="macro")
f2 = fbeta_score(y_test, y_pre, beta=2, average="binary")
p1 = precision_score(y_test, y_pre, average="macro")
r1 = recall_score(y_test, y_pre, average="macro")
print("accuracy score : ",a1)
print("f1 score : ",f1)
print("f2 score : ",f2)
print("precision score : ",p1)
print("recall score : ",r1)

# Computing Confusion matrix for the above algorithm

cnf_matrix = confusion_matrix(y_test, y_pre)
np.set_printoptions(precision=2)


In [ ]:
roc_auc = roc_auc_score(y_test, dectree_basic.predict_proba(X_test_selected)[:,1])
cf_matrix_roc_auc(dectree_basic, y_test, dectree_basic.predict(X_test_selected), dectree_basic.predict_proba(X_test_selected)[:,1], feature_importances=True)

Using Reduced features with lgbm classifier

In [ ]:
def train_models(model, X_train_selected, X_test_selected, y_train, y_test):

    model.fit(X_train_selected, y_train)
    output = {
      'AUC': roc_auc_score(y_test, model.predict_proba(X_test_selected)[:,1]),
      'Accuracy': accuracy_score(y_test, model.predict(X_test_selected)),
      'Precision': precision_score(y_test, model.predict(X_test_selected)),
      'Recall': recall_score(y_test, model.predict(X_test_selected)),
      'F1': f1_score(y_test, model.predict(X_test_selected)),
      'F2': fbeta_score(y_test, model.predict(X_test_selected), beta=2, average='binary')

      }
          
    return output

In [ ]:
%%time

name = []
auc = []
accuracy = []
precision = []
recall = []
f1 = []
f2 = []
time_ = []

for model in models:
    start = time.time()
    results = train_models(model, X_train_selected, X_test_selected, y_train, y_test)

    name.append(type(model).__name__)
    auc.append(results['AUC'])
    accuracy.append(results['Accuracy'])
    precision.append(results['Precision'])
    recall.append(results['Recall'])
    f1.append(results['F1'])
    f2.append(results['F2'])
    time_.append(time.time()-start)

#Initialise data of lists
models_rfe = pd.DataFrame(data=[name, auc, accuracy, precision, recall, f1, f2, time_]).T
models_rfe.columns = ['Model', 'AUC', 'Accuracy', 'Precision', 'Recall', 'F1', 'F2','Time']
models_rfe.sort_values('AUC', ascending=False, inplace=True)

In [ ]:
#Models performance with features selection
base_models_sm


In [ ]:
###Reminder before… features selection
base_models

In [ ]:
###Reminder before… features selection
models_rfe

In [ ]:
for model in models:
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test_selected)[:,1])
    cf_matrix_roc_auc(model, y_test, model.predict(X_test_selected), model.predict_proba(X_test_selected)[:,1], feature_importances=True)

## Fonction de Cout


In [ ]:
#Tests fonctionels avec des listes contenant 4 int: tp, tn, fp, fn
test_0 = [500, 300, 10, 30]
test_1 = [500, 300, 30, 10]
test_2 = [400, 300, 70, 50]
test_3 = [400, 300, 50, 70]
test_4 = [350, 250, 80, 120]
test_5 = [350, 250, 180, 90]

tests = [test_0, test_1, test_2, test_3, test_4, test_5]


def my_score(predictions_success: list) -> int:
    '''scoring fonction'''
    tp, tn, fp, fn = predictions_success
  
    beta = 3
    recall = tp / (tp + fn)
    precision = tp / (tp + fp) 
    fscore = (1+beta)*(tp / ((1+3)*tp + beta*fn + fp))
    
    return 1-fscore

for i, t in enumerate(tests):
    print("############################")
    print("Test", i, ":", t)
    print("Score :", my_score(t))

In [ ]:
def custom_score(y_true, y_pred, beta=3) :
    '''function penalize fp and fn…'''
    tp, tn, fp, fn = confusion_matrix(y_true, y_pred).ravel()

    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    fscore = (1+beta)*((tp / ((1+beta)*tp + beta*fn + fp)))

    return 1-fscore

In [ ]:
def cost_false(model, y_true, X_true):
    '''Cost function analyzes prediction errors False Pos. and False Neg.'''
    cm = confusion_matrix(y_true, model.predict(X_true))
    FP = cm[0][1]/np.sum(cm)
    FN = cm[1][0]/np.sum(cm)
  
    print("False Pos: {0:.2%}".format(FP))
    print("False Neg: {0:.2%}".format(FN))
    return FP, FN

## Hyperparameters tunning - Hyperopt et AUC score

In [ ]:
#!pip install networkx
#!pip install hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
#Parameter space
space = {
    'n_estimators': hp.quniform('n_estimators', 500, 1000, 500),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
    'max_depth': hp.quniform('max_depth', 5, 10, 3),
    'subsample': hp.uniform('subsample', 0.60, 0.95),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.60, 0.95),
    'reg_lambda': hp.uniform('reg_lambda', 1, 20)
    }

In [ ]:
def objective(params, eval_metric='custom_score'):
    #objective function to be minimized. 
    #Hyperopt will seek to minimize the loss returned by this function.
    
    params = {
        'n_estimators': int(params['n_estimators']),
        'learning_rate': params['learning_rate'],
        'max_depth': int(params['max_depth']),
        'subsample': params['subsample'],
        'colsample_bytree': params['colsample_bytree'],
        'reg_lambda': params['reg_lambda'],
        'device':'gpu'
        }
      
    model= LGBMClassifier(**params)
    cv = StratifiedKFold(5)
    if eval_metric == 'roc_auc': 
        score = cross_val_score(model, X_train_selected, y_train, scoring='roc_auc', cv=cv).mean()

    elif eval_metric == 'custom_score': 
        y_pred = cross_val_predict(model, X_train_selected, y_train, method='predict', cv=cv)
        #y_pred = cross_val_predict(model, X_train, y_train, method='predict', cv=cv)
        score = custom_score(y_train, y_pred)
  
    loss = 1 - score    
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

In [ ]:
%%time
best = fmin(fn=objective, space=space, max_evals=10, rstate=np.random.default_rng(1), algo=tpe.suggest)


In [ ]:
%%time
lgbm = LGBMClassifier( #Fit a new model based on the best parameters
    n_estimators=int(best['n_estimators']), 
    colsample_bytree= best['colsample_bytree'],
    learning_rate= best['learning_rate'],
    max_depth= int(best['max_depth']),
    subsample= best['subsample'],
    reg_lambda= best['reg_lambda'],
    device='gpu')

lgbm.fit(X_train_selected, y_train)
#lgbm.fit(X_train, y_train)
pickle.dump(lgbm, open("LGBMClassifier.pkl", "wb"))

In [ ]:
roc_auc = roc_auc_score(y_test, lgbm.predict_proba(X_test_selected)[:,1])
#roc_auc = roc_auc_score(y_test, lgbm.predict_proba(X_test)[:,1])
print('AUC : %0.6f' %roc_auc)

In [ ]:
cf_matrix_roc_auc(lgbm, y_test, lgbm.predict(X_test_selected), lgbm.predict_proba(X_test_selected)[:,1], feature_importances=True)
#cf_matrix_roc_auc(lgbm, y_test, lgbm.predict(X_test), lgbm.predict_proba(X_test)[:,1], feature_importances=True)

In [ ]:
#FP and FN
cost_false(lgbm, y_test, X_test_selected)

In [ ]:
#Check list features importance 
sorted_idx = np.argsort(lgbm.feature_importances_)[::-1]
for index in sorted_idx:
    print([X_data.columns[index], lgbm.feature_importances_[index]]) 

In [ ]:
def objective(params, eval_metric='roc_auc'):
  #objective function to be minimized. 
  #Hyperopt will seek to minimize the loss returned by this function.

    params = {
        'n_estimators': int(params['n_estimators']),
        'learning_rate': params['learning_rate'],
        'max_depth': int(params['max_depth']),
        'subsample': params['subsample'],
        'colsample_bytree': params['colsample_bytree'],
        'reg_lambda': params['reg_lambda'],
        'device':'gpu'}
  
    
    model= LGBMClassifier(**params)
    cv = StratifiedKFold(5)
    if eval_metric == 'roc_auc': 
        score = cross_val_score(model, X_train_selected, y_train, scoring='roc_auc', cv=cv).mean()

    elif eval_metric == 'custom_score': 
        y_pred = cross_val_predict(model, X_train_selected, y_train, method='predict', cv=cv)
        score = custom_score(y_train, y_pred)
  
    loss = 1 - score    
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

In [ ]:
%%time
best = fmin(fn=objective, space=space, max_evals=10, rstate=np.random.default_rng(1), algo=tpe.suggest)


In [ ]:
%%time
lgbm = LGBMClassifier( #Fit a new model based on the best parameters
    n_estimators=int(best['n_estimators']), 
    colsample_bytree= best['colsample_bytree'],
    learning_rate= best['learning_rate'],
    max_depth= int(best['max_depth']),
    subsample= best['subsample'],
    reg_lambda= best['reg_lambda'],
    device='gpu')

lgbm.fit(X_train_selected, y_train)


In [ ]:
roc_auc = roc_auc_score(y_test, lgbm.predict_proba(X_test_selected)[:,1])
print('AUC : %0.6f' %roc_auc)

In [ ]:
cf_matrix_roc_auc(lgbm, y_test, lgbm.predict(X_test_selected), lgbm.predict_proba(X_test_selected)[:,1], feature_importances=True)
